In [0]:
dbutils.widgets.dropdown("environment", "dev", 
                         ["dev", "prod"], "Environment")

In [0]:
env = dbutils.widgets.get("environment")

In [0]:
!pip install msal 

In [0]:
import msal
import requests
import json
import time

In [0]:
def get_token_interactive(scope):
    """
    Get token via interative authentication without persisting credentials. Can be used when MFA is configured in the account.
    """
    app = msal.PublicClientApplication(client_id, authority=authority_url)
    result = app.acquire_token_interactive(scope)
    if 'access_token' in result:
        return(result['access_token'])
    else:
        print('Error in get_token_interactive:',result.get("error"), result.get("error_description"))



def get_token_username_password(scope):
    """
    Get token with username and password without interaction.
    """
    app = msal.PublicClientApplication(client_id, authority=authority_url)
    result = app.acquire_token_by_username_password(username=username,password=password,scopes=scope)
    if 'access_token' in result:
        return(result['access_token'])
    else:
        print('Error in get_token_username_password:',result.get("error"), result.get("error_description"))


def get_token_for_client(scope,client_id,authority_url,client_secret):
    """
    Get token for authenticating via client/client secret
    Input: 
        scope: power bi scope for calling APIs followed by the word ./default (usually, "https://analysis.windows.net/powerbi/api./default")
        client_id: Application ID of the service principal used to call the API
        authority_url: "https://login.microsoftonline.com/common/<domain>"
        client_secret: secret value created under the Service principal used for authentication
    Return: 
        Authorization token
    """
    app = msal.ConfidentialClientApplication(client_id,authority=authority_url,client_credential=client_secret)
    result = app.acquire_token_for_client(scopes=scope)
    if 'access_token' in result:
        return(result['access_token'])
    else:
        print(result)
        print('Error in get_token_for_client:',result.get("error"), result.get("error_description"))


In [0]:
 def get_dataset():
    response = requests.get(f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceID}/datasets',
                       headers={
                           'Authorization': f'Bearer {access_token}',
                           'Content-type': 'application/json'}
                      )
    
    print(response)

    return [dataset for dataset in response.json()['value']] if response else None

In [0]:
databricks_secret_scope = f"phg-data-{env}-dbsp-dbxbackend"
client_key = f"phg-data-{env}-scrt-pbiclientid"
secret_key = f"phg-data-{env}-scrt-pbisecret"

# Azure service principal Application (client) ID 
client_id = dbutils.secrets.get(scope = databricks_secret_scope, key = client_key) 

# Azure service principal client secret
client_secret = dbutils.secrets.get(scope = databricks_secret_scope, key = secret_key)


In [0]:
#Power BI datasets on Amica Analytics - Development workspace
authority_url = 'https://login.microsoftonline.com/henrymwuamica.onmicrosoft.com'
scope =['https://analysis.windows.net/powerbi/api/.default']

datasets = {"PHG Report Pace Visual":"63982fb6-162f-492b-8394-155c6ac7e473",
            "PHG Report Embed": "ae5a5b3f-b549-4624-ada9-e7bea3d1fcef"}

access_token = get_token_for_client(scope,client_id=client_id,authority_url=authority_url,client_secret=client_secret)

workspaceID = "21a8d045-5b71-4fd8-886b-8251515eb32b"
dataflow_url = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspaceID + '/dataflows'
header = {'Authorization': f'Bearer {access_token}','Content-Type':'application/json'}


for name,dataset in datasets.items():

    refresh_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceID}/datasets/{dataset}/refreshes'
    print(refresh_url)
    response = requests.post(refresh_url,
                         headers={
                             'Authorization': f'Bearer {access_token}',
                             'Content-type': 'application/json'
                         }
                        )
    if response.ok:
        print(f"Successfully ran the refresh task for {name} : {dataset}")
        
    else:
        print(f"Encountered an issue in the refresh task")
        print(f"Status: {response.reason}")
        print(f"{response.json()}")
    
    time.sleep(5*60)

In [0]:
#Power BI datasets on Amica Analytics - QA workspace

authority_url = 'https://login.microsoftonline.com/henrymwuamica.onmicrosoft.com'
scope =['https://analysis.windows.net/powerbi/api/.default']

databricks_secret_scope = f"phg-data-{env}-dbsp-dbxbackend"
client_key = f"phg-data-{env}-scrt-qapbiclientid"
secret_key = f"phg-data-{env}-scrt-qapbisecret"

# Azure service principal Application (client) ID 
client_id = dbutils.secrets.get(scope = databricks_secret_scope, key = client_key) 

# Azure service principal client secret
client_secret = dbutils.secrets.get(scope = databricks_secret_scope, key = secret_key)

access_token = get_token_for_client(scope,client_id=client_id,authority_url=authority_url,client_secret=client_secret)

datasets = {"PHG Report Pace Visual":"11de615e-1f93-415d-a62b-2b3f2692fd66",
            "PHG Report Embed": "82b51de4-024f-4336-83ea-8a8a4816c100"}

workspaceID = "a330008a-afe4-433f-9599-e7f513275f17"
dataflow_url = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspaceID + '/dataflows'
header = {'Authorization': f'Bearer {access_token}','Content-Type':'application/json'}


for name,dataset in datasets.items():

    refresh_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceID}/datasets/{dataset}/refreshes'
    print(refresh_url)
    response = requests.post(refresh_url,
                         headers={
                             'Authorization': f'Bearer {access_token}',
                             'Content-type': 'application/json'
                         }
                        )
    if response.ok:
        print(f"Successfully ran the refresh task for {name} : {dataset}")
        
    else:
        print(f"Encountered an issue in the refresh task")
        print(f"Status: {response.reason}")
        print(f"{response.json()}")
    
    time.sleep(5*60)

In [0]:
#Power BI datasets on Amica Analytics workspace

authority_url = 'https://login.microsoftonline.com/henrymwuamica.onmicrosoft.com'
scope =['https://analysis.windows.net/powerbi/api/.default']

databricks_secret_scope = f"phg-data-{env}-dbsp-dbxbackend"
client_key = f"phg-data-{env}-scrt-pbiclientid"
secret_key = f"phg-data-{env}-scrt-pbisecret"

# Azure service principal Application (client) ID 
client_id = dbutils.secrets.get(scope = databricks_secret_scope, key = client_key) 

# Azure service principal client secret
client_secret = dbutils.secrets.get(scope = databricks_secret_scope, key = secret_key)

access_token = get_token_for_client(scope,client_id=client_id,authority_url=authority_url,client_secret=client_secret)

datasets = {"PHG Report Pace Visual":"dd6abe6f-d9e3-4d6d-a77f-17b3fb3c27bb",
            "PHG Report Embed": "e3057552-0411-4ec7-8352-ea77834a22a7"}

workspaceID = "d5b0f556-eaf8-4f47-86d2-860e88a869d6"
dataflow_url = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspaceID + '/dataflows'
header = {'Authorization': f'Bearer {access_token}','Content-Type':'application/json'}


for name,dataset in datasets.items():

    refresh_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceID}/datasets/{dataset}/refreshes'
    print(refresh_url)
    response = requests.post(refresh_url,
                         headers={
                             'Authorization': f'Bearer {access_token}',
                             'Content-type': 'application/json'
                         }
                        )
    if response.ok:
        print(f"Successfully ran the refresh task for {name} : {dataset}")
        
    else:
        print(f"Encountered an issue in the refresh task")
        print(f"Status: {response.reason}")
        print(f"{response.json()}")
    
    time.sleep(5*60)